In [37]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import numpy as np

import os

load_dotenv()

PG_STRING = os.getenv("BITIO_PG_STRING")
BOOKLIST_TABLE = '"bitdotio/best books 2021"."list_of_lists"'
MEDIA_TABLE = '"bitdotio/best books 2021"."media_lists"'
CELEB_TABLE = '"bitdotio/best books 2021"."individual_lists"'
RANK_TABLE = '"bitdotio/best books 2021"."book_ranks"'

# Analysis of End-of-Year Book Lists

Find the data [here](https://bit.io/bitdotio/best%20books%202021#).

# The Data
The data can be found in [this bit.io repository](https://bit.io/bitdotio/best%20books%202021#). Other good sources for aggregate end-of-year lists include:
- [yearendlists.com](https://www.yearendlists.com/) which also includes lists for TV, music, movies, and more.
- [The Ultimate Best Books of 2021 List (Lithub)](https://lithub.com/the-ultimate-best-books-of-2021-list/) which aggregates many year-end lists to obtain a "definitive" year-end list.

## How We Got the Data
We looked at all of the book lists for 2021 on [yearendlists.com](https://www.yearendlists.com/) to identify the lists. We followed a few key principles in selecting sources:
- Look for Discriminating Sources: the lists included ten books or fewer (the publications have to narrow the books to a list of favorites. a [list of 100](https://time.com/collection/100-must-read-books-2021/) doesn't help us in finding a consensus "best book.")
- Use High-profile sources: we didn't have specific criteria for this one, but in general, we looked for lists from well-known individuals or media sources, not from any blog with a book list we could find on the Internet.
- Prefer generality: We looked for "best books" list, not "best fiction" or "best science fiction" or other sub-classifications of books. In one case (TIME), we took both the "top fiction" and "top nonfiction" lists rather than omitting the publication entirely.
- Avoid redundancy: Use only a single list (or, in the case of TIME, two non-overlapping lists) per source. We don't want to count a single book more than once for a given source.

## Obtain the data from bit.io

In [7]:
# Helper Fucntion for Downloading Datasets
def download_dataset(target, pg_string):
    engine = create_engine(pg_string)
    # SQL for querying an entire table
    sql = f"""
        SELECT *
        FROM {target};
    """
    # Return SQL query as a pandas dataframe
    with engine.connect() as conn:
        # Set 1 minute statement timeout (units are milliseconds)
        conn.execute("SET statement_timeout = 60000;")
        df = pd.read_sql(sql, conn)
    return df

df_media = download_dataset(MEDIA_TABLE, PG_STRING)
df_individual = download_dataset(CELEB_TABLE, PG_STRING)
df_rank = download_dataset(RANK_TABLE, PG_STRING)

In [10]:
df_media = df_media.merge(df_rank, how="left", left_on="title", right_on="title").rename(columns={'count':'rank'})
df_individual = df_individual.merge(df_rank, how="left", left_on="title", right_on="title").rename(columns={'count':'rank'})

df_media

,source,title,author,link,rank
0,Vulture,Dear Senthuran: A Black Spirit Memoir,Akwaeke Emezi,https://www.vulture.com/article/best-books-of-...,1
1,Wall Street Journal,Robert E. Lee: A Life,Allen C. Guelzo,https://www.wsj.com/articles/the-10-best-books...,1
2,Barnes & Noble,Call Us What We Carry,Amanda Gorman,https://www.barnesandnoble.com/b/books/barnes-...,1
3,TIME,"Invisible Child: Poverty, Survival & Hope in a...",Andrea Elliott,https://time.com/6125895/best-nonfiction-books...,2
4,New York Times,"Invisible Child: Poverty, Survival & Hope in a...",Andrea Elliott,https://www.nytimes.com/2021/11/30/books/revie...,2
...,...,...,...,...,...
160,New York Times,The Copenhagen Trilogy: Childhood; Youth; Depe...,Tove Ditlevsen,https://www.nytimes.com/2021/11/30/books/revie...,3
161,NPR Fresh Air,We Run the Tides,Vendela Vida,https://www.npr.org/2021/12/13/1060830513/maur...,1
162,Slate,The Committed,Viet Thanh Nguyen,https://slate.com/culture/2021/12/best-books-2...,1
163,New York Public Library,All Her Little Secrets,Wanda M. Morris,https://www.nypl.org/blog/2021/11/23/introduci...,1


In [21]:
# List Appearing Once
df_media['title_author'] = df_media.title.values + ' (' + df_media.author.values + ')'
once = (df_media
.loc[df_media['rank']==1, :]
.loc[:,'title_author']
.values
)
", ".join(once)

"Dear Senthuran: A Black Spirit Memoir (Akwaeke Emezi), Robert E. Lee: A Life (Allen C. Guelzo), Call Us What We Carry (Amanda Gorman), On Juneteenth (Annette Gordon-Reed), Heard-Hoard (Atsuro Riley), The War for Gloria (Atticus Lish), A Touch of Jen (Beth Morgan), The Rock Eaters (Brenda Peynado), Burnt Toast and Other Disasters: A Book of Heroic Hacks, Fabulous Fixes, and Secret Sauces (Cal Peternell), Open Water (Caleb Azumah Nelson), Mama Phife Represents (Cheryl Boyce-Taylor), The Life of the Mind (Christine Smallwood), I Love You But I've Chosen Darkness (Claire Vaye Watkins), The Afghanistan Papers: A Secret History of the War (Craig Whitlock), The Kissing Bug: A True Story of a Family, an Insect, and a Nation's Neglect of a Deadly Disease (Daisy Hernández), Wayward (Dana Spiotta), An Inventory of Losses (Judith Schalansky), You Will Get through This Night (Daniel Howell), One Friday in April: A Story of Suicide and Survival (Donald Antrim), The Strategy of Denial: American Defe

In [57]:
names = []
props = []
counts = []

groups = df_media.groupby('source')
for name, group in groups:
    print(name)
    others = df_media.loc[df_media['source'] != name]
    length = group.shape[0]
    titles = group['title']
    num = titles.isin(others['title'])
    n_times = (others['title'].isin(titles)).sum()
    counts.append(n_times)
    props.append(num.mean())
    names.append(name)

pd.DataFrame({'source':names, 'prop':props, 'times':counts}).sort_values('prop', ascending=False)

Barnes & Noble
Chicago Public Library
Chicago Tribune
Entertainment Weekly
KCRW
Los Angeles Times
NPR Fresh Air
New York Public Library
New York Times
Publishers Weekly
Slate
TIME
The Philadelphia Inquirer
Vulture
Wall Street Journal
Washington Post


In [62]:
names_i = []
props_i = []
counts_i = []

groups = df_individual.groupby('source')
for name, group in groups:
  #  print(name)
    others = df_media.loc[df_media['source'] != name]
    length = group.shape[0]
    titles = group['title']
    num = titles.isin(others['title'])
    n_times = (others['title'].isin(titles)).sum()
    counts_i.append(n_times)
    props_i.append(num.mean())
    names_i.append(name)

pd.DataFrame({'source':names_i, 'prop':props_i, 'times':counts_i}).sort_values('prop', ascending=False)



,source,prop,times
0,Barack Obama,0.692308,33
1,Bill Gates,0.200000,5


0.6923076923076923